# lab-working-with-api

### Instructions

#### Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. Do it for all the flights between two dates that cost the same.

#### some of the sources are not available in the APIs, so we are not be able to finish the lab with the exact instructions
#### we will create a free lab

In [1]:
# copying python request from the site with APIs https://aerodatabox.p.rapidapi.com
import requests
import pandas as pd

url = "https://aerodatabox.p.rapidapi.com/health/services/feeds/FlightSchedules/airports"

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': "9613ca6ca5msh62659872159916fp1a0a2cjsndd9a3e6ff324"
    }

response = requests.get(url, headers=headers)

print(response.text)

{"items":["KSAF","PAEN","UUOB","EGLL","KCAK","KITH","LEGE","OEHL","ZBYC","ZSSH","HKKI","KSGF","KTVC","USRK","VYKI","CYYJ","LGKJ","PASH","VIPT","WIII","CYZR","FALA","FTTY","KBFF","KMIA","SBVH","WMKN","YMAE","ZGGG","CYFC","KFAT","OISS","SKPD","ENLK","KGLH","LFRZ","NZWB","RJBB","RKTH","SBTC","VOGO","ZYJZ","CYXS","ENGM","EPWR","OIZC","PAPB","ROAH","UKLL","VVPB","BGPT","GVSF","HKMO","SAZS","YFTZ","CCZ2","KDSM","NTTR","PAUK","UNEE","WBKL","ZBTL","EGPN","KBED","KBZN","LFRD","PAKP","SVMC","DTTJ","EDDF","LEJR","MBSY","NFNA","NTTO","PAJZ","SCSE","UUYY","YMTG","AYMS","LTCR","MYGF","SBDB","WBGQ","DIOD","ENFL","KCRW","KJMS","LTDA","MYSM","NTGQ","SBTT","DTNH","ENHV","KCLE","MBGT","OIBL","SBPV","VIJP","YIVL","ZUZH","CAP3","CYSM","MHSC","OEDF","RJKI","RPMH","SECU","YWBS","KSLC","LEMI","LTFE","SKCC","ZLZY","CYWJ","KFLL","LIMF","MDJB","NZCH","PFKU","UAUU","KSNA","LEBZ","LICB","MMPG","PASA","SWTS","VLSK","WIDN","ZHNY","PAWI","RJCC","SWGN","VYTL","CYGW","FAPE","KBJI","KDAL","KYKM","UWPS","CYHR","EYKA","RP

In [2]:
# we will check the option search 'airports by free text' and we will get a python code like the following
url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

querystring = {"q":"Thessaloniki","limit":"5"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': "9613ca6ca5msh62659872159916fp1a0a2cjsndd9a3e6ff324"
    }

response = requests.get(url, headers=headers, params=querystring)
response.json()
# we get a json file, which is actually a dictionary and lists but it has some differences with the dictionaries
# for instance in transforming a dictionary to a dataframe or a json file to a dataframe
# Thessaloniki has only one airport this is the reason we get only one nested dictionary

{'items': [{'icao': 'LGTS',
   'iata': 'SKG',
   'name': 'Thessaloniki, Thessaloniki Macedonia',
   'shortName': 'Macedonia',
   'municipalityName': 'Thessaloniki',
   'location': {'lat': 40.5197, 'lon': 22.9709},
   'countryCode': 'GR'}]}

In [3]:
# getting the key of the outside dictionary
response.json().keys() 

dict_keys(['items'])

In [4]:
# getting the values of the outside dictionary
response.json().values() 

dict_values([[{'icao': 'LGTS', 'iata': 'SKG', 'name': 'Thessaloniki, Thessaloniki Macedonia', 'shortName': 'Macedonia', 'municipalityName': 'Thessaloniki', 'location': {'lat': 40.5197, 'lon': 22.9709}, 'countryCode': 'GR'}]])

In [5]:
#  getting the keys of the nested dictionary
response.json()['items'][0].keys() 

dict_keys(['icao', 'iata', 'name', 'shortName', 'municipalityName', 'location', 'countryCode'])

In [6]:
# Let's get an element of the dictionary, let's try iata
response.json()['items'][0]['iata']

'SKG'

In [7]:
# Let's get an other element of the dictionary, let's try shortName

In [8]:
response.json()['items'][0]['shortName']

'Macedonia'

In [9]:
# response.json()['items'][100]
# in case we use an index out of range we will get an error

In [10]:
# creating a function
# we use the same headers
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': "9613ca6ca5msh62659872159916fp1a0a2cjsndd9a3e6ff324"
    }


def get_codes():
    
    location = input('Insert the location:')
    print()
    
    url = "https://aerodatabox.p.rapidapi.com/airports/search/term"
    querystring = {"q":location.lower(), "limit":"5"}

    response = requests.get(url, headers=headers, params=querystring)
    codes = response.json()['items'] if 'items' in response.json() else location

    return codes

# in case we have more than one airports in a city, in order to return icao
def choose_code():
    
    codes = get_codes()
    
    while type(codes) is str or len(codes) != 1:
        
        if not codes:
            print('Try another location. Unable to find code for:', codes, '\n')
        else:
            print('More than one location found. Insert one of the following airports:', [code['shortName'] for code in codes], '\n')
        
        codes = get_codes()
    
    return codes[0]['icao']


def flight_distance_time(departure, arrival):
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{departure}/distance-time/{arrival}"
    response = requests.get(url, headers=headers)
    return pd.json_normalize(response.json()).T



In [11]:
print('[DEPARTURE]')
departure = choose_code()
print('[DESTINATION]')
arrival = choose_code()

flight_df = flight_distance_time(departure, arrival)
flight_df

[DEPARTURE]
Insert the location:Thessaloniki

[DESTINATION]
Insert the location:New York

More than one location found. Insert one of the following airports: ['La Guardia', 'John F Kennedy'] 

Insert the location:John F Kennedy

More than one location found. Insert one of the following airports: ['John F Kennedy', 'John F Kennedy Memorial'] 

Insert the location:John F Kennedy

More than one location found. Insert one of the following airports: ['John F Kennedy', 'John F Kennedy Memorial'] 

Insert the location:John F Kennedy Memorial



,0
message,You have exceeded the MONTHLY quota for Tier 2...


In [12]:
# Let's try different stuff
NBA = response1 = requests.get("https://api.sportsdata.io/api/nba/fantasy/json/CurrentSeason")
NBA.status_code
# we are not properly authenticated

401

In [13]:
# getting APIs for earthquakes in USA
from pandas.io.json import json_normalize
response2 = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson')
out = response2.json()
earthquake_data = json_normalize(out['features'])
earthquake_data

C:\Users\Admin\AppData\Local\Temp/ipykernel_8120/3792849529.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  earthquake_data = json_normalize(out['features'])


,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,nc73764170,2.10,"9km NNE of Lake Pillsbury, CA",1659645852070,1659645951253,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",nearby-cities,origin,phase-data,",15.0,0.192300,0.21,75.0,md,earthquake,"M 2.1 - 9km NNE of Lake Pillsbury, CA",Point,"[-122.928833, 39.4884987, -0.28]"
1,Feature,ak0229xk6c3l,1.40,"9 km WNW of Tatitlek, Alaska",1659645087697,1659645214861,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.62,NaN,ml,earthquake,"M 1.4 - 9 km WNW of Tatitlek, Alaska",Point,"[-146.8448, 60.8919, 27.4]"
2,Feature,ak0229xk5uzn,1.20,"131 km SE of McCarthy, Alaska",1659644945959,1659645220400,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.34,NaN,ml,earthquake,"M 1.2 - 131 km SE of McCarthy, Alaska",Point,"[-141.2157, 60.5868, 28.8]"
3,Feature,ak0229xk5n0c,1.30,"124 km SE of McCarthy, Alaska",1659644882526,1659644997724,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.59,NaN,ml,earthquake,"M 1.3 - 124 km SE of McCarthy, Alaska",Point,"[-141.3562, 60.6124, 7.6]"
4,Feature,ak0229xk3l1s,1.70,"124 km SE of McCarthy, Alaska",1659644324000,1659644487774,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.56,NaN,ml,earthquake,"M 1.7 - 124 km SE of McCarthy, Alaska",Point,"[-141.3665, 60.608, 8.8]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8967,Feature,ci40297520,0.32,"12km N of Cabazon, CA",1657056241270,1657130561264,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",nearby-cities,origin,phase-data,scitech-link,",13.0,0.031800,0.11,102.0,ml,earthquake,"M 0.3 - 12km N of Cabazon, CA",Point,"[-116.7703333, 34.0285, 13.02]"
8968,Feature,ak0228jykkws,1.10,"13 km NNE of Susitna, Alaska",1657055911962,1657056035658,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,1.28,NaN,ml,earthquake,"M 1.1 - 13 km NNE of Susitna, Alaska",Point,"[-150.4145, 61.653, 49.4]"
8969,Feature,nc73754991,0.30,"2km NNE of The Geysers, CA",1657055038110,1657056316271,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",nearby-cities,origin,phase-data,scitech-link,",12.0,0.008054,0.03,94.0,md,earthquake,"M 0.3 - 2km NNE of The Geysers, CA",Point,"[-122.7505035, 38.7970009, 2.56]"
8970,Feature,ak0228jy7ps4,1.20,"35 km WNW of Anchor Point, Alaska",1657054709802,1657054868227,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.39,NaN,ml,earthquake,"M 1.2 - 35 km WNW of Anchor Point, Alaska",Point,"[-152.4016, 59.9215, 70.5]"


In [14]:
# getting the maximum of the feature properties.mag
earthquake_data['properties.mag'].max()

7.0